In [1]:
import pyetrade

consumer_key = "a694bf19a1a7ee1724e35a21e626bffe"
consumer_secret = "c52c35186ec76bcbe37e308e2e37d942e179dc07921efb3c79094b525785dd25"

oauth = pyetrade.ETradeOAuth(consumer_key, consumer_secret)
print(oauth.get_request_token())  # Use the printed URL

verifier_code = input("Enter verification code: ")
tokens = oauth.get_access_token(verifier_code)

print("\nCopy the following 4 values into the manager's arguments:")
print(consumer_key)
print(consumer_secret)
print(tokens["oauth_token"])
print(tokens["oauth_token_secret"])

https://us.etrade.com/e/t/etws/authorize?key=a694bf19a1a7ee1724e35a21e626bffe&token=i2UL5+wEaPACizYfgZ9TOhIg8a0ESbJjCFGkxVlHcVk=


Enter verification code:  O9HNR



Copy the following 4 values into the manager's arguments:
a694bf19a1a7ee1724e35a21e626bffe
c52c35186ec76bcbe37e308e2e37d942e179dc07921efb3c79094b525785dd25
sz5O4NgKEAe7MyTW08lwElcgn6xB7dAc9b25t1MaquI=
3plbiNXks9P6gTqgr6nQBAyT/uS0xnHL//TSRi86r+I=


In [1]:
import options_manager
import logging
import datetime

logging.basicConfig(level=logging.INFO)

manager = options_manager.OptionsManager(
    consumer_key="a694bf19a1a7ee1724e35a21e626bffe", 
    consumer_secret="c52c35186ec76bcbe37e308e2e37d942e179dc07921efb3c79094b525785dd25", 
    oauth_token="sz5O4NgKEAe7MyTW08lwElcgn6xB7dAc9b25t1MaquI=",
    oauth_secret="3plbiNXks9P6gTqgr6nQBAyT/uS0xnHL//TSRi86r+I="
)

ModuleNotFoundError: No module named 'options_chains'

In [2]:
manager.get_market_price("GOOG")

2979.01

In [12]:
manager.get_options_info(
    ticker="GOOG", 
    percent_below_mkt=.05,
    percent_above_mkt=.1, # all arguments below have defaults and don't need to be passed
    increment=100,
    month_look_ahead=3,
    hide_no_contracts=True,
    hide_no_interest=True,
    contracts_to_buy=1, # defaults to max available
)

INFO:options_manager:Restricting strike price to (2830, 3276) for 2979.01 market price.
INFO:options_manager:Restricting search to 3 valid expiry dates.
INFO:options_manager:Found 12 options for specified expiry dates and strikes.
INFO:options_manager:Hiding 3 puts due to no volume.
INFO:options_manager:Returning 9 valid options.


[{'optionType': 'PUT',
  'strikePrice': '2900.0',
  'symbol': 'GOOG',
  'bid': '14.2',
  'ask': '15.2',
  'volume': '89',
  'openInterest': '1764',
  'lastPrice': '14.53',
  'OptionGreeks': OrderedDict([('rho', '-0.173800'),
               ('vega', '1.474400'),
               ('theta', '-1.713400'),
               ('delta', '-0.219100'),
               ('gamma', '0.002500'),
               ('iv', '0.233400'),
               ('currentValue', 'true')]),
  'expiryDate': datetime.date(2021, 11, 19),
  'marketPrice': 2979.01,
  'belowMarketPct': 0.026522233896495887,
  'auxiliaryInfo': {'contractsToBuy': 1,
   'revenue': 1470.0,
   'annualizedRevenue': 53655.0,
   'annualizedReturn': 0.18501724137931036}},
 {'optionType': 'PUT',
  'strikePrice': '3000.0',
  'symbol': 'GOOG',
  'bid': '47.7',
  'ask': '54.1',
  'volume': '38',
  'openInterest': '66',
  'lastPrice': '61.99',
  'OptionGreeks': OrderedDict([('rho', '-0.412400'),
               ('vega', '1.964700'),
               ('theta', '-2.